# PyCaret Clustering — Kaggle
For Kaggle: use **Add data** to attach a dataset. CSV files appear under `/kaggle/input/...`.
Run the first cell to install PyCaret 3.3.2.


Assignment Done by :- **Dev Mulchandani**

In [1]:
%pip -q install -U pip
%pip -q install 'pycaret==3.3.2'
import sys
print('Python:', sys.version)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.6 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.9.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
google-colab 1.0.0 requires google-auth==2.38.0, but you have google-auth 2.40.3 which is incompatible.
google-colab 1.0.0 requires notebook==6.5.7, but you have notebook 6.5.4 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.1.4 which is incompatible.
google-colab 1.0.0

In [2]:
import glob, pandas as pd
csvs = sorted([p for p in glob.glob('/kaggle/input/**/*.csv', recursive=True)])
if not csvs:
    raise SystemExit('No CSVs found under /kaggle/input. Click **Add data** and attach your dataset, then rerun this cell.')
for i,p in enumerate(csvs):
    print(f'{i}: {p}')
idx = int(input('Enter the index of the CSV to load: '))
DATA_PATH = csvs[idx]
print('Using:', DATA_PATH)
data = pd.read_csv(DATA_PATH)
print('Shape:', data.shape)
display(data.head())

0: /kaggle/input/clustering-dataset/Mall_Customers.csv


Enter the index of the CSV to load:  0


Using: /kaggle/input/clustering-dataset/Mall_Customers.csv
Shape: (200, 5)


,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


In [3]:
# --- Run FIRST: force CPU & avoid RAPIDS/GPU backends ---
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""  # hide GPUs so GPU paths are skipped
# If RAPIDS bits are present, remove them so PyCaret won't try to load them
!pip -q uninstall -y cuml cudf cuml-cu12 cudf-cu12 cupy-cuda12x cupy-cuda11x cupy || true


In [4]:
from pycaret.clustering import *
exp = setup(data, normalize=True, session_id=123)
model = create_model('kmeans')
assigned = assign_model(model)
display(assigned.head())
plot_model(model, plot='cluster')
save_model(model, 'best_clustering_model')

,Description,Value
0,Session id,123
1,Original data shape,"(200, 5)"
2,Transformed data shape,"(200, 5)"
3,Numeric features,4
4,Categorical features,1
5,Preprocess,True
6,Imputation type,simple
7,Numeric imputation,mean
8,Categorical imputation,mode
9,Maximum one-hot encoding,-1


,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.3013,70.1301,1.3133,0,0,0


,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100),Cluster
0,1,Male,19,15,39,Cluster 3
1,2,Male,21,15,81,Cluster 3
2,3,Female,20,16,6,Cluster 1
3,4,Female,23,16,77,Cluster 1
4,5,Female,31,17,40,Cluster 1


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['CustomerID', 'Age',
                                              'Annual Income (k$)',
                                              'Spending Score (1-100)'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=['Gender'],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('ordinal_encoding',
                  TransformerWrapper(include=['Gender'],
                                     transformer=OrdinalEncoder(cols=['Gender'],
                                                                handle_missing='return_nan',
                                                                mapping=[{'col': 'Gender',
                                                                          'data_type': dtype('O